# Segmenting and Clustering Neighborhoods in Toronto

## Applied Data Science Capstone

Part I is to create a dataframe of the postal codes, boroughs, and neighborhoods of Toronto.<br>
Part II is to append the latitude and longitude of each postal code region.<br>
Part III is to cluster the regions using venue data from Foursquare.

#### Rohan Lewis

#### 2020.10.28

### Packages

In [1]:
import folium
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import pgeocode
import requests
from sklearn.cluster import KMeans

### Foursquare Developer Setup

In [2]:
#Read spreadsheet.
df_foursquare = pd.read_excel("Foursquare.xlsx")

#Extract credentials.
CLIENT_ID = df_foursquare.iloc[0][0]
CLIENT_SECRET = df_foursquare.iloc[0][1]
VERSION = df_foursquare.iloc[0][2]

#Global Constants
RADIUS = 700
LIMIT = 100

## I. Dataframe from Wikipedia.

In [3]:
#URL.
toronto_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

#Read table.
df = pd.read_html(toronto_wiki)[0]

#Remove Postal Codes with Boroughs that are "Not assigned".
df_toronto = df[df['Borough'] != "Not assigned"].reset_index(drop = True)
df_toronto.columns = ['Postal Code', 'Borough', 'Neighborhood']
df_toronto

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
df_toronto.shape

(103, 3)

## II. Latitude and Longitude

In [5]:
#Set country to Canada.
canada = pgeocode.Nominatim('ca')

#Initialize empty columns.
df_toronto['Latitude'] = None
df_toronto['Longitude'] = None

#Iterate through postal codes, set latitude and longitude value.
for idx, pc in enumerate(df_toronto["Postal Code"]) :
    location = canada.query_postal_code(pc)
    df_toronto.at[idx, 'Latitude'] = location.latitude
    df_toronto.at[idx, 'Longitude'] = location.longitude

In [6]:
#Check for missed values.
df_toronto[df_toronto['Latitude'].isnull()]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
76,M7R,Mississauga,Canada Post Gateway Processing Centre,NaN,NaN


List of all assigned Postal Codes in Toronto, with their respective Boroughs, Neighborhoods, and Coordinates.

In [7]:
#Manual entry.
df_toronto.at[76, 'Latitude'] = 43.6368
df_toronto.at[76,'Longitude'] = -79.6204
df_toronto

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.33
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6518,-79.5076
99,M4Y,Downtown Toronto,Church and Wellesley,43.6656,-79.383
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.7804,-79.2505
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939


## III. Neighborhood Clustering

### 1. Functions

#### A. Request URL for api.foursquare.com.

In [8]:
def api_url(lat, lng):
    temp = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'
    return(temp.format(CLIENT_ID,
                       CLIENT_SECRET,
                       VERSION,
                       lat,
                       lng,
                       RADIUS,
                       LIMIT))

#### B. Retrieve venues from neighborhoods.

In [9]:
def get_nearby_venues(names, latitudes, longitudes):
    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        #Create the API request URL.
        url = api_url(lat, lng)
        
        #Make the GET request.
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #Return only relevant information for each nearby venue.
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                             'Neighborhood Latitude',
                             'Neighborhood Longitude', 
                             'Venue', 
                             'Venue Latitude', 
                             'Venue Longitude',
                             'Venue Category']
    
    return(nearby_venues)

#### C. Most Common Venues by Neighborhood.

In [10]:
def return_most_common_venues(row):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:10]

#### D. Neighborhoods by Number of Restaurants

In [11]:
#Filters a dataframe based on a list of neighborhoods.
#In Part III, each tier is a list of neighborhoods with a specific number of restaurants.
#In the Appendix, each tier is a cluster.
def nh_filter(df, tier):
    temp = df[df['Neighborhood'].isin(tier)]    
    return(temp)

### 2. Restaurants in Toronto by Neighborhood

In [12]:
#Retrieve all venues in Toronto by Neighbourhood.
toronto_venues = get_nearby_venues(names = df_toronto['Neighborhood'],
                                   latitudes = df_toronto['Latitude'],
                                   longitudes = df_toronto['Longitude'])

#"Restaurant" was not complete as a keyword to identify all restaurants.
#I added several other keywords to create a more complete list of eateries.
restaurant_list = [venue for venue in toronto_venues['Venue Category'].unique().tolist()
                   if any(cat in venue for cat in ["Breakfast",
                                                   "Creperie",
                                                   "Gastropub"
                                                   "Joint",
                                                   "Noodle",
                                                   "Place",
                                                   "Restaurant"])]

#Complete list of restaurants in Toronto by Neighbourhood.  See Appendix for details.
toronto_restaurants = toronto_venues[toronto_venues['Venue Category'].isin(restaurant_list)].reset_index(drop = True)

#Create a column for each type of restaurant.
toronto_onehot = pd.get_dummies(toronto_restaurants[['Venue Category']], prefix = "", prefix_sep = "")

#Neighbourhood name is the first column.
toronto_onehot.insert(0, "Neighborhood", toronto_restaurants['Neighborhood'], True)

#Group by Neighbourhood and calculate the percent occurence for each restaurant type.  
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

In [13]:
temp = toronto_restaurants['Neighborhood'].value_counts().to_frame(name = "Count")
#Create tier lists.
nh_5_or_more = temp[temp['Count'] > 4].index.tolist()
nh_3_4 = temp[(temp['Count'].isin([3,4]))].index.tolist()
nh_1_2 = temp[(temp['Count'].isin([1,2]))].index.tolist()

Determine the top 10 restaurant categories for each neighborhood.

In [15]:
ordinals = ['st', 'nd', 'rd']

#Create columns according to number of top venues.
columns = ['Neighborhood']
for ind in np.arange(10):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, ordinals[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

#Create a dataframe of neighborhoods with a sorted order of the top 10 types of eateries.
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :])

### 3. Cluster Neighbourhoods

Neighbourhoods were grouped into 10 clusters, by restaurant type. See **IV. Cluster and Neighborhood Details**.

In [16]:
temp_df = nh_filter(toronto_grouped, nh_5_or_more)
temp_df2 = nh_filter(neighborhoods_venues_sorted, nh_5_or_more)
temp_df3 = nh_filter(neighborhoods_venues_sorted, nh_3_4)
temp_df4 = nh_filter(neighborhoods_venues_sorted, nh_1_2)


#Run k-means clustering
kmeans = KMeans(n_clusters = 8, random_state = 0).fit(temp_df.drop('Neighborhood', 1))

#Add clustering labels.
temp_df2.insert(0, 'Cluster', kmeans.labels_)
#Neighborhoods with 3 or 4 restaurants are assigned to Cluster 8.
temp_df3.insert(0, 'Cluster', 8)
#Neighborhoods with 1 or 2 restaurants are assigned to Cluster 9.
temp_df4.insert(0, 'Cluster', 9)
toronto_merged = df_toronto

#Merge to add latitude/longitude for each neighbourhood in Toronto.
toronto_merged = toronto_merged.join(temp_df2.append(temp_df3).append(temp_df4).set_index('Neighborhood'),
                                     on = 'Neighborhood')

### 4. Neighbourhoods in Toronto without Restaurants.
An 11th cluster was added for neighbourhoods without restaurants.

In [18]:
##Neighborhoods with no restaurants are assigned to Cluster 10.
np.nan_to_num(toronto_merged['Cluster'], copy = False, nan = 10)

#Cluster is changed to int.
toronto_merged['Cluster'] = toronto_merged['Cluster'].astype(int)

### 5. Map of Boroughs in Toronto by Restaurant Cluster

Map Setup

In [19]:
bold_colors = ['#1ca42b', '#ffe119', '#800080', '#c69edf', '#ffffff', '#e6194b', '#0000af',
               '#9a6324', '#46f0f0', '#808080', '#000000', '#fabebe', '#aaffc3', '#f032e6']

address = 'Toronto, CN'

geolocator = Nominatim(user_agent = "toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Map

In [20]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location = [latitude + 0.07, longitude], zoom_start = 11)

# add markers to map
for lat, lng, label, cluster in zip(toronto_merged['Latitude'],
                                    toronto_merged['Longitude'],
                                    toronto_merged['Neighborhood'],
                                    toronto_merged['Cluster']):
    
    html = "<b>Neighborhood(s):</b><br>" + label + "<br><br><b>Cluster: </b>" + str(cluster)
    iframe = folium.IFrame(html = html, width = 300, height = 100)

    #label = folium.Popup(label + '\n\n\n\nCluster: ' + str(cluster), parse_html = True, width = 300)
    folium.CircleMarker([lat, lng],
                        radius = 10,
                        popup = folium.Popup(iframe),
                        color = bold_colors[cluster],
                        fill = True,
                        fill_color = bold_colors[cluster],
                        fill_opacity = 0.8,
                        parse_html = False).add_to(map_toronto) 
map_toronto

## IV. Cluster and Neighborhood Details

### 1. Restaurants by Neighborhood, Category, and Venue

#### Complete Information.

In [21]:
pd.set_option('display.max_rows', None)
toronto_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant
1,Victoria Village,43.7276,-79.3148,Pizza Nova,43.725824,-79.312860,Pizza Place
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,Sukhothai,43.658444,-79.365681,Thai Restaurant
4,"Regent Park, Harbourfront",43.6555,-79.3626,Impact Kitchen,43.656369,-79.356980,Restaurant
5,"Regent Park, Harbourfront",43.6555,-79.3626,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant
6,"Regent Park, Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
7,"Regent Park, Harbourfront",43.6555,-79.3626,Fusaro's,43.653347,-79.369517,Italian Restaurant
8,"Regent Park, Harbourfront",43.6555,-79.3626,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
9,"Regent Park, Harbourfront",43.6555,-79.3626,El Catrin,43.650601,-79.358920,Mexican Restaurant


#### Restaurant Category Count

In [22]:
toronto_restaurants['Venue Category'].value_counts().to_frame()

,Venue Category
Restaurant,105
Pizza Place,88
Sandwich Place,66
Italian Restaurant,64
Japanese Restaurant,58
Sushi Restaurant,43
American Restaurant,40
Fast Food Restaurant,39
Breakfast Spot,39
Thai Restaurant,35


#### Restaurant Franchise/Name Count

In [23]:
toronto_restaurants['Venue'].value_counts().to_frame()

,Venue
Subway,40
Pizza Pizza,22
McDonald's,11
KFC,8
A&W,7
Pizza Hut,7
Ki Modern Japanese + Bar,7
Pizzeria Libretto,7
John & Sons Oyster House,7
Chotto Matte,7


### 2. Clusters

#### Cluster Filter Function

In [24]:
#Returns neighborhoods in a cluster.
#Returns restaurants in neighborhoods in cluster.
def cluster_filter(cluster):
    cluster_df = toronto_merged[toronto_merged["Cluster"] == cluster]
    restaurant_df = toronto_restaurants[toronto_restaurants["Neighborhood"].isin(cluster_df['Neighborhood'].tolist())]
    restaurant_count = ("Cluster " + str(cluster) + " has " + str(restaurant_df.shape[0]) +
                        " out of the " + str(toronto_restaurants.shape[0]) +" restaurants listed in Toronto.")
    return(cluster_df, restaurant_count, restaurant_df)

#### Cluster 0

These neighborhoods have a variety and large number of restaurants. 

In [25]:
c0 = cluster_filter(0)
c0[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,0,Sandwich Place,Pizza Place,Sushi Restaurant,Japanese Restaurant,Mediterranean Restaurant,Restaurant,Ethiopian Restaurant,French Restaurant,Indian Restaurant,Creperie
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783,0,Japanese Restaurant,Ramen Restaurant,Middle Eastern Restaurant,Falafel Restaurant,Breakfast Spot,Restaurant,Sushi Restaurant,Sandwich Place,Pizza Place,Poke Place
14,M4C,East York,Woodbine Heights,43.6913,-79.3116,0,Pizza Place,Ethiopian Restaurant,Sushi Restaurant,Thai Restaurant,Breakfast Spot,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
23,M4G,East York,Leaside,43.7124,-79.3644,0,Restaurant,Sandwich Place,Mexican Restaurant,Indian Restaurant,Breakfast Spot,Sushi Restaurant,German Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant
24,M5G,Downtown Toronto,Central Bay Street,43.6564,-79.386,0,Italian Restaurant,Ramen Restaurant,Thai Restaurant,Breakfast Spot,Sushi Restaurant,Vegetarian / Vegan Restaurant,Restaurant,Modern European Restaurant,Portuguese Restaurant,Falafel Restaurant
28,M3H,North York,"Bathurst Manor, Wilson Heights, Downsview North",43.7535,-79.4472,0,Mediterranean Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Pizza Place,Restaurant,Sandwich Place,Brazilian Restaurant,Breakfast Spot,Japanese Restaurant,Italian Restaurant
34,M3J,North York,"Northwood Park, York University",43.7694,-79.4921,0,Caribbean Restaurant,Middle Eastern Restaurant,Pizza Place,Chinese Restaurant,Sandwich Place,Japanese Restaurant,American Restaurant,Italian Restaurant,Indian Restaurant,Indian Chinese Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.6803,-79.3538,0,Greek Restaurant,Italian Restaurant,Restaurant,Pizza Place,Sushi Restaurant,Falafel Restaurant,Fast Food Restaurant,Cuban Restaurant,New American Restaurant,Caribbean Restaurant
59,M2N,North York,"Willowdale, Willowdale East",43.7673,-79.4111,0,Fast Food Restaurant,Pizza Place,Korean Restaurant,Ramen Restaurant,Japanese Restaurant,Sandwich Place,Restaurant,Sushi Restaurant,Vietnamese Restaurant,Seafood Restaurant
67,M4P,Central Toronto,Davisville North,43.7135,-79.3887,0,Greek Restaurant,Breakfast Spot,Pizza Place,Sushi Restaurant,Sandwich Place,Middle Eastern Restaurant,Japanese Restaurant,Indian Restaurant,Indian Chinese Restaurant,Hong Kong Restaurant


In [26]:
print(c0[1])
c0[2]

Cluster 0 has 389 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
30,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Tokyo Sushi,43.665885,-79.386977,Sushi Restaurant
31,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Como En Casa,43.665160,-79.384796,Mexican Restaurant
32,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Sansotei Ramen 三草亭,43.666735,-79.385353,Ramen Restaurant
33,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,T-Swirl Crepe,43.663452,-79.384125,Creperie
34,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Nando's,43.661728,-79.386391,Portuguese Restaurant
35,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Mercatto,43.660391,-79.387664,Italian Restaurant
36,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Crown Princess Fine Dining 伯爵名宴,43.666455,-79.387698,Chinese Restaurant
37,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Ethiopian House,43.666599,-79.385669,Ethiopian Restaurant
38,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Kothur Indian Cuisine,43.667872,-79.385659,Indian Restaurant
39,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889,Storm Crow Manor,43.666840,-79.381593,Theme Restaurant


#### Cluster 1

These three neighborhoods seem to have primarily American, Fast Food, and general Restaurants.

In [27]:
c1 = cluster_filter(1)
c1[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,1,Fast Food Restaurant,Greek Restaurant,American Restaurant,Restaurant,Mediterranean Restaurant,Sushi Restaurant,Seafood Restaurant,Breakfast Spot,Burrito Place,Jewish Restaurant
33,M2J,North York,"Fairview, Henry Farm, Oriole",43.7801,-79.3479,1,Fast Food Restaurant,Restaurant,American Restaurant,Asian Restaurant,Japanese Restaurant,Burrito Place,Chinese Restaurant,Hawaiian Restaurant,Filipino Restaurant,French Restaurant
88,M8V,Etobicoke,"New Toronto, Mimico South, Humber Bay Shores",43.6075,-79.5013,1,Restaurant,American Restaurant,Mexican Restaurant,Fast Food Restaurant,Pizza Place,Vietnamese Restaurant,Greek Restaurant,Filipino Restaurant,French Restaurant,German Restaurant


In [28]:
print(c1[1])
c1[2]

Cluster 1 has 27 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
18,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,RH Courtyard Café,43.724874,-79.455536,Restaurant
19,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,JOEY,43.724131,-79.454042,American Restaurant
20,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,Villa Madina,43.724684,-79.454172,Mediterranean Restaurant
21,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,Jimmy The Greek,43.726138,-79.452106,Greek Restaurant
22,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,The Burger's Priest,43.720789,-79.456766,Fast Food Restaurant
23,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,Yuki Japanese Restaurant,43.720610,-79.456119,Sushi Restaurant
24,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,Krystos,43.718516,-79.455855,Greek Restaurant
25,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,The Cheesecake Factory,43.726297,-79.454262,American Restaurant
26,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,Red Lobster,43.718952,-79.456245,Seafood Restaurant
27,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504,KFC,43.716600,-79.447000,Fast Food Restaurant


#### Cluster 2

These neighborhoods have a variety and large number of restaurants.  Many of these neighborhoods are located in or near downtown, on the Lake Ontario side.

**Cluster 0** and **Cluster 2** have about 70% of all listed restaurants in Toronto.

In [29]:
c2 = cluster_filter(2)
c2[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,M5C,Downtown Toronto,St. James Town,43.6513,-79.3756,2,Japanese Restaurant,Restaurant,Seafood Restaurant,American Restaurant,Italian Restaurant,Pizza Place,French Restaurant,New American Restaurant,Creperie,Comfort Food Restaurant
20,M5E,Downtown Toronto,Berczy Park,43.6456,-79.3754,2,Restaurant,Japanese Restaurant,Italian Restaurant,Creperie,American Restaurant,Seafood Restaurant,Vegetarian / Vegan Restaurant,French Restaurant,Molecular Gastronomy Restaurant,New American Restaurant
30,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.6496,-79.3833,2,Restaurant,Japanese Restaurant,Breakfast Spot,Italian Restaurant,American Restaurant,Asian Restaurant,Seafood Restaurant,Latin American Restaurant,Vegetarian / Vegan Restaurant,New American Restaurant
37,M6J,West Toronto,"Little Portugal, Trinity",43.648,-79.4177,2,Restaurant,Italian Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant,Greek Restaurant,New American Restaurant,Vietnamese Restaurant,Breakfast Spot,Seafood Restaurant,Korean Restaurant
42,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.6469,-79.3823,2,Restaurant,Seafood Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Sushi Restaurant,Mediterranean Restaurant,Greek Restaurant,French Restaurant,New American Restaurant
43,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.6383,-79.4301,2,Restaurant,Sandwich Place,Indian Restaurant,Breakfast Spot,Italian Restaurant,Mexican Restaurant,Hawaiian Restaurant,North Indian Restaurant,French Restaurant,Pizza Place
48,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.6492,-79.3823,2,Restaurant,Japanese Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Breakfast Spot,Italian Restaurant,Seafood Restaurant,Creperie,Latin American Restaurant,New American Restaurant
54,M4M,East Toronto,Studio District,43.6561,-79.3406,2,American Restaurant,Vietnamese Restaurant,Taco Place,Italian Restaurant,Middle Eastern Restaurant,Seafood Restaurant,Comfort Food Restaurant,Chinese Restaurant,Latin American Restaurant,Thai Restaurant
87,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.6404,-79.3995,2,Restaurant,Italian Restaurant,Japanese Restaurant,Sandwich Place,Asian Restaurant,Caribbean Restaurant,Ramen Restaurant,Seafood Restaurant,Latin American Restaurant,Greek Restaurant
92,M5W,Downtown Toronto,Stn A PO Boxes,43.6437,-79.3787,2,Restaurant,Japanese Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Sushi Restaurant,Italian Restaurant,Greek Restaurant,French Restaurant,Salad Place,Seafood Restaurant


In [30]:
print(c2[1])
c2[2]

Cluster 2 has 324 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
113,St. James Town,43.6513,-79.3756,Gyu-Kaku Japanese BBQ,43.651422,-79.375047,Japanese Restaurant
114,St. James Town,43.6513,-79.3756,GEORGE Restaurant,43.653346,-79.374445,Restaurant
115,St. James Town,43.6513,-79.3756,Crepe TO,43.650063,-79.374587,Creperie
116,St. James Town,43.6513,-79.3756,Mystic Muffin,43.652484,-79.372655,Middle Eastern Restaurant
117,St. James Town,43.6513,-79.3756,Terroni,43.650927,-79.375602,Italian Restaurant
118,St. James Town,43.6513,-79.3756,Richmond Station,43.651569,-79.379266,American Restaurant
119,St. James Town,43.6513,-79.3756,NAMI,43.650853,-79.375887,Japanese Restaurant
120,St. James Town,43.6513,-79.3756,The Carbon Bar,43.653367,-79.374965,Restaurant
121,St. James Town,43.6513,-79.3756,The Chase,43.650952,-79.379422,New American Restaurant
122,St. James Town,43.6513,-79.3756,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant


#### Cluster 3

*Thorncliffe Park* and *India Bazaar/The Beaches West* have a large concentration of Indian Restaurants.  9 out of the 29 Indian Restaurants listed in Toronto are located in these two neighborhoods.

In [31]:
c3 = cluster_filter(3)
c3[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
29,M4H,East York,Thorncliffe Park,43.7059,-79.3464,3,Indian Restaurant,Afghan Restaurant,Turkish Restaurant,Sandwich Place,Restaurant,Fast Food Restaurant,Middle Eastern Restaurant,Pizza Place,Greek Restaurant,Falafel Restaurant
47,M4L,East Toronto,"India Bazaar, The Beaches West",43.6693,-79.3155,3,Indian Restaurant,Sandwich Place,Italian Restaurant,Fast Food Restaurant,Restaurant,Pizza Place,Asian Restaurant,Indian Chinese Restaurant,Burrito Place,Sushi Restaurant


In [32]:
print(c3[1])
c3[2]

Cluster 3 has 34 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
261,Thorncliffe Park,43.7059,-79.3464,Bamiyan Kabob,43.707824,-79.343446,Afghan Restaurant
262,Thorncliffe Park,43.7059,-79.3464,Iqbal Kebab & Sweet Centre,43.705923,-79.351521,Indian Restaurant
263,Thorncliffe Park,43.7059,-79.3464,Pasha Authentic Turkish Cuisine,43.708240,-79.342699,Turkish Restaurant
264,Thorncliffe Park,43.7059,-79.3464,Faley Restaurant,43.708113,-79.343579,Indian Restaurant
265,Thorncliffe Park,43.7059,-79.3464,A&W,43.706275,-79.344670,Fast Food Restaurant
266,Thorncliffe Park,43.7059,-79.3464,Subway,43.704596,-79.349670,Sandwich Place
267,Thorncliffe Park,43.7059,-79.3464,Pizza Pizza,43.705564,-79.347139,Pizza Place
268,Thorncliffe Park,43.7059,-79.3464,Hakka Garden,43.704578,-79.349770,Indian Restaurant
269,Thorncliffe Park,43.7059,-79.3464,Afghan Cuisine,43.708075,-79.342985,Afghan Restaurant
270,Thorncliffe Park,43.7059,-79.3464,Swiss Chalet,43.707786,-79.344132,Restaurant


#### Cluster 4

*Moore Park/Summerhill East* was clustered as an individual entity.

In [33]:
c4 = cluster_filter(4)
c4[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,M4T,Central Toronto,"Moore Park, Summerhill East",43.6899,-79.3853,4,Thai Restaurant,Seafood Restaurant,Japanese Restaurant,Sandwich Place,German Restaurant,Eastern European Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant


In [34]:
print(c4[1])
c4[2]

Cluster 4 has 5 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
763,"Moore Park, Summerhill East",43.6899,-79.3853,New Thai Restaurant,43.688869,-79.391163,Thai Restaurant
764,"Moore Park, Summerhill East",43.6899,-79.3853,Mermaid Fish & Grill,43.688680,-79.392063,Seafood Restaurant
765,"Moore Park, Summerhill East",43.6899,-79.3853,Suki Japanese Cuisine,43.688668,-79.391854,Japanese Restaurant
766,"Moore Park, Summerhill East",43.6899,-79.3853,Subway,43.688698,-79.392031,Sandwich Place
767,"Moore Park, Summerhill East",43.6899,-79.3853,Thai Express,43.688400,-79.393096,Thai Restaurant


#### Cluster 5

This cluster seems to have a large proportion of Italian Restaurants.  1/4 of all Italian (not Pizza) Restaurants listed in Toronto are located here.

In [35]:
c5 = cluster_filter(5)
c5[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626,5,Italian Restaurant,Thai Restaurant,Breakfast Spot,Restaurant,Mediterranean Restaurant,Spanish Restaurant,Mexican Restaurant,Indian Restaurant,German Restaurant,French Restaurant
31,M6H,West Toronto,"Dufferin, Dovercourt Village",43.6655,-79.4378,5,Mexican Restaurant,Vietnamese Restaurant,Middle Eastern Restaurant,Portuguese Restaurant,Pizza Place,Italian Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant,African Restaurant,Indian Restaurant
55,M5M,North York,"Bedford Park, Lawrence Manor East",43.7335,-79.4177,5,Italian Restaurant,Sandwich Place,Restaurant,Pizza Place,Comfort Food Restaurant,Sushi Restaurant,Greek Restaurant,Thai Restaurant,Indian Restaurant,American Restaurant
69,M6P,West Toronto,"High Park, The Junction South",43.6605,-79.4633,5,Italian Restaurant,Mexican Restaurant,Thai Restaurant,Vietnamese Restaurant,Cajun / Creole Restaurant,Fast Food Restaurant,Seafood Restaurant,Soup Place,Sushi Restaurant,Sandwich Place
73,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.7143,-79.4065,5,Restaurant,Italian Restaurant,Chinese Restaurant,Ramen Restaurant,Mexican Restaurant,Thai Restaurant,Fast Food Restaurant,Pizza Place,Falafel Restaurant,Filipino Restaurant
74,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.6736,-79.4035,5,Italian Restaurant,Sandwich Place,Mexican Restaurant,Jewish Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Modern European Restaurant,Pizza Place,French Restaurant
79,M4S,Central Toronto,Davisville,43.702,-79.3853,5,Pizza Place,Sandwich Place,Italian Restaurant,Fast Food Restaurant,Indian Restaurant,Mexican Restaurant,Thai Restaurant,Seafood Restaurant,Restaurant,Sushi Restaurant


In [36]:
print(c5[1])
c5[2]

Cluster 5 has 94 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,"Regent Park, Harbourfront",43.6555,-79.3626,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.6555,-79.3626,Sukhothai,43.658444,-79.365681,Thai Restaurant
4,"Regent Park, Harbourfront",43.6555,-79.3626,Impact Kitchen,43.656369,-79.356980,Restaurant
5,"Regent Park, Harbourfront",43.6555,-79.3626,Mangia and Bevi Resto-Bar,43.652250,-79.366355,Italian Restaurant
6,"Regent Park, Harbourfront",43.6555,-79.3626,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
7,"Regent Park, Harbourfront",43.6555,-79.3626,Fusaro's,43.653347,-79.369517,Italian Restaurant
8,"Regent Park, Harbourfront",43.6555,-79.3626,Souk Tabule,43.653756,-79.354390,Mediterranean Restaurant
9,"Regent Park, Harbourfront",43.6555,-79.3626,El Catrin,43.650601,-79.358920,Mexican Restaurant
10,"Regent Park, Harbourfront",43.6555,-79.3626,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
11,"Regent Park, Harbourfront",43.6555,-79.3626,FAMO Sandwiches,43.654064,-79.369515,Sandwich Place


#### Cluster 6

Korean!  *Christie* and *Willowdale/Newtonbrook* have over 70% of all listed Korean restaurants in Toronto.

In [37]:
c6 = cluster_filter(6)
c6[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,M6G,Downtown Toronto,Christie,43.6683,-79.4205,6,Korean Restaurant,Mexican Restaurant,Indian Restaurant,Pizza Place,Chinese Restaurant,Latin American Restaurant,Italian Restaurant,Ethiopian Restaurant,Ramen Restaurant,Restaurant
52,M2M,North York,"Willowdale, Newtonbrook",43.7915,-79.4103,6,Korean Restaurant,Middle Eastern Restaurant,Pizza Place,Sandwich Place,Sushi Restaurant,Japanese Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant


In [38]:
print(c6[1])
c6[2]

Cluster 6 has 40 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
223,Christie,43.6683,-79.4205,Banjara Indian Cuisine,43.662916,-79.421911,Indian Restaurant
224,Christie,43.6683,-79.4205,Tacos El Asador,43.663751,-79.416954,Taco Place
225,Christie,43.6683,-79.4205,Sunrise House Korean Restaurant 해뜨는집,43.664183,-79.415746,Korean Restaurant
226,Christie,43.6683,-79.4205,Buk Chang Dong Soon Tofu,43.663842,-79.417093,Korean Restaurant
227,Christie,43.6683,-79.4205,Tofu Village 두부마을 豆腐村,43.663943,-79.416652,Korean Restaurant
228,Christie,43.6683,-79.4205,Vinny’s Panini,43.670679,-79.426148,Italian Restaurant
229,Christie,43.6683,-79.4205,Apiecalypse Now! Vegan Pizza & Snack Bar,43.663399,-79.418651,Pizza Place
230,Christie,43.6683,-79.4205,Napoli Centrale,43.669391,-79.413028,Pizza Place
231,Christie,43.6683,-79.4205,Yummy Korean Restaurant,43.664431,-79.413937,Korean Restaurant
232,Christie,43.6683,-79.4205,Actinolite,43.667858,-79.428054,Restaurant


#### Cluster 7

Fast Food and Pizza Places seem to be popular in these neighborhoods.

In [39]:
c7 = cluster_filter(7)
c7[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,M6B,North York,Glencairn,43.7081,-79.4479,7,Pizza Place,Mediterranean Restaurant,Fast Food Restaurant,Japanese Restaurant,Latin American Restaurant,Belgian Restaurant,Asian Restaurant,Korean Restaurant,Jewish Restaurant,African Restaurant
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.7678,-79.1866,7,Pizza Place,Fast Food Restaurant,Breakfast Spot,Greek Restaurant,Restaurant,Mexican Restaurant,Indian Restaurant,Sandwich Place,Vietnamese Restaurant,Falafel Restaurant
40,M3K,North York,Downsview,43.739,-79.4692,7,Fast Food Restaurant,Pizza Place,Vietnamese Restaurant,Falafel Restaurant,American Restaurant,Sandwich Place,Caribbean Restaurant,Greek Restaurant,Filipino Restaurant,French Restaurant
46,M3L,North York,Downsview,43.7334,-79.5116,7,Fast Food Restaurant,Pizza Place,Vietnamese Restaurant,Falafel Restaurant,American Restaurant,Sandwich Place,Caribbean Restaurant,Greek Restaurant,Filipino Restaurant,French Restaurant
53,M3M,North York,Downsview,43.7319,-79.4928,7,Fast Food Restaurant,Pizza Place,Vietnamese Restaurant,Falafel Restaurant,American Restaurant,Sandwich Place,Caribbean Restaurant,Greek Restaurant,Filipino Restaurant,French Restaurant
60,M3N,North York,Downsview,43.7568,-79.521,7,Fast Food Restaurant,Pizza Place,Vietnamese Restaurant,Falafel Restaurant,American Restaurant,Sandwich Place,Caribbean Restaurant,Greek Restaurant,Filipino Restaurant,French Restaurant
82,M1T,Scarborough,"Clarks Corners, Tam O'Shanter, Sullivan",43.7812,-79.3036,7,Pizza Place,Sandwich Place,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Thai Restaurant,Noodle House,Ethiopian Restaurant,Falafel Restaurant,Filipino Restaurant
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.7432,-79.5876,7,Pizza Place,Sandwich Place,Fast Food Restaurant,Caribbean Restaurant,Vietnamese Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant
90,M1W,Scarborough,"Steeles West, L'Amoreaux West",43.8016,-79.3216,7,Fast Food Restaurant,Chinese Restaurant,Indian Restaurant,Breakfast Spot,Sandwich Place,Pizza Place,Vietnamese Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant


In [40]:
print(c7[1])
c7[2]

Cluster 7 has 48 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
99,Glencairn,43.7081,-79.4479,Columbus Colombian Bakery (Panaderia Colombiana),43.709173,-79.453331,Latin American Restaurant
100,Glencairn,43.7081,-79.4479,Miyako Sushi Restaurant,43.709111,-79.443930,Japanese Restaurant
101,Glencairn,43.7081,-79.4479,Camarra's Pizzeria & Restaurant,43.708329,-79.453488,Pizza Place
102,Glencairn,43.7081,-79.4479,McDonald's,43.704919,-79.452238,Fast Food Restaurant
103,Glencairn,43.7081,-79.4479,Darosa,43.706970,-79.453121,Mediterranean Restaurant
104,Glencairn,43.7081,-79.4479,Domino's Pizza,43.707170,-79.442658,Pizza Place
145,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Little Caesars Pizza,43.769046,-79.184386,Pizza Place
146,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Sunset Grill,43.772319,-79.184900,Breakfast Spot
147,"Guildwood, Morningside, West Hill",43.7678,-79.1866,Swiss Chalet,43.768122,-79.190493,Restaurant
148,"Guildwood, Morningside, West Hill",43.7678,-79.1866,KFC,43.768900,-79.185600,Fast Food Restaurant


#### Cluster 8

These neighborhoods each have only 3 or 4 listed restaurants.

In [41]:
c8 = cluster_filter(8)
c8[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094,8,Pizza Place,Fast Food Restaurant,Breakfast Spot,Vietnamese Restaurant,Hawaiian Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hong Kong Restaurant
38,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.7298,-79.2639,8,Sandwich Place,Chinese Restaurant,Pizza Place,Vietnamese Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant
70,M9P,Etobicoke,Westmount,43.6949,-79.5323,8,Sandwich Place,Chinese Restaurant,Pizza Place,Vietnamese Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant
72,M2R,North York,"Willowdale, Willowdale West",43.7786,-79.445,8,Pizza Place,Sandwich Place,Latin American Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Vietnamese Restaurant
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.6898,-79.5582,8,Chinese Restaurant,American Restaurant,Pizza Place,Sandwich Place,Vietnamese Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant
93,M8W,Etobicoke,"Alderwood, Long Branch",43.6021,-79.5402,8,Pizza Place,Sandwich Place,Latin American Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Vietnamese Restaurant
94,M9W,Etobicoke,"Northwest, West Humber - Clairville",43.7144,-79.5909,8,Mediterranean Restaurant,Pizza Place,Swiss Restaurant,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6325,-79.4939,8,Fast Food Restaurant,Italian Restaurant,Eastern European Restaurant,Vietnamese Restaurant,Greek Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Hong Kong Restaurant


In [42]:
print(c8[1])
c8[2]

Cluster 8 has 26 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
63,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094,Pizza Pizza,43.705159,-79.313130,Pizza Place
64,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094,Harvey's,43.710964,-79.309085,Fast Food Restaurant
65,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094,Rise & Dine Eatery,43.705769,-79.311638,Breakfast Spot
66,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094,Venice Pizza,43.705921,-79.313957,Pizza Place
365,"Kennedy Park, Ionview, East Birchmount Park",43.7298,-79.2639,Subway,43.731471,-79.268692,Sandwich Place
366,"Kennedy Park, Ionview, East Birchmount Park",43.7298,-79.2639,Chung Moi,43.732328,-79.269327,Chinese Restaurant
367,"Kennedy Park, Ionview, East Birchmount Park",43.7298,-79.2639,241 Pizza,43.731789,-79.271275,Pizza Place
608,Westmount,43.6949,-79.5323,Mayflower Chinese Food,43.692753,-79.531566,Chinese Restaurant
609,Westmount,43.6949,-79.5323,Subway,43.692927,-79.531471,Sandwich Place
610,Westmount,43.6949,-79.5323,Pizza Hut,43.696431,-79.533233,Pizza Place


#### Cluster 9

These neighborhoods each have only 1 or 2 listed restaurants.

In [43]:
c9 = cluster_filter(9)
c9[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,M4A,North York,Victoria Village,43.7276,-79.3148,9,Portuguese Restaurant,Pizza Place,Vietnamese Restaurant,Latin American Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant
6,M1B,Scarborough,"Malvern, Rouge",43.8113,-79.193,9,Fast Food Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hong Kong Restaurant,Eastern European Restaurant
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.6505,-79.5517,9,Pizza Place,Vietnamese Restaurant,Malay Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.6437,-79.5767,9,Pizza Place,Vietnamese Restaurant,Malay Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant
19,M4E,East Toronto,The Beaches,43.6784,-79.2941,9,Sandwich Place,Malay Restaurant,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Vietnamese Restaurant
21,M6E,York,Caledonia-Fairbanks,43.6889,-79.4507,9,Mexican Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hong Kong Restaurant
26,M1H,Scarborough,Cedarbrae,43.7686,-79.2389,9,Thai Restaurant,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hong Kong Restaurant
27,M2H,North York,Hillcrest Village,43.8015,-79.3577,9,Chinese Restaurant,Vietnamese Restaurant,Eastern European Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant,Hawaiian Restaurant
32,M1J,Scarborough,Scarborough Village,43.7464,-79.2323,9,Restaurant,Indian Restaurant,Vietnamese Restaurant,Greek Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Hawaiian Restaurant
35,M4J,East York,"East Toronto, Broadview North (Old East York)",43.6872,-79.3368,9,Italian Restaurant,Breakfast Spot,Vietnamese Restaurant,Hawaiian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant,French Restaurant,German Restaurant,Greek Restaurant


In [44]:
print(c9[1])
c9[2]

Cluster 9 has 23 out of the 1010 restaurants listed in Toronto.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Victoria Village,43.7276,-79.3148,Portugril,43.725819,-79.312785,Portuguese Restaurant
1,Victoria Village,43.7276,-79.3148,Pizza Nova,43.725824,-79.312860,Pizza Place
62,"Malvern, Rouge",43.8113,-79.1930,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
105,"West Deane Park, Princess Gardens, Martin Grov...",43.6505,-79.5517,Pizza Pizza,43.648755,-79.548726,Pizza Place
106,"West Deane Park, Princess Gardens, Martin Grov...",43.6505,-79.5517,Pizza Nova,43.649092,-79.549000,Pizza Place
144,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.6437,-79.5767,Pizza Hut,43.641845,-79.576556,Pizza Place
158,The Beaches,43.6784,-79.2941,Subway,43.680716,-79.287006,Sandwich Place
188,Caledonia-Fairbanks,43.6889,-79.4507,Rebozos,43.684773,-79.449588,Mexican Restaurant
253,Cedarbrae,43.7686,-79.2389,Thai One On,43.774468,-79.241268,Thai Restaurant
254,Hillcrest Village,43.8015,-79.3577,New Greattime Corp.,43.807414,-79.356717,Chinese Restaurant


#### Cluster 10

These neighborhoods each have no listed restaurants.

In [45]:
c10 = cluster_filter(10)
c10[0]

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.7545,-79.33,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,M3B,North York,Don Mills,43.745,-79.359,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.7878,-79.1564,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,M3C,North York,Don Mills,43.7334,-79.3329,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,M6C,York,Humewood-Cedarvale,43.6915,-79.4307,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,M1G,Scarborough,Woburn,43.7712,-79.2144,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.623,-79.3936,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.7122,-79.2843,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,M2L,North York,"York Mills, Silver Hills",43.7547,-79.3764,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## V. Concluding Thoughts.

I am extremely pleased that **Cluster 3** is clearly Indian and **Cluster 7** is clearly Korean.  To a lesser extent, I am ok with **Cluster 1** and **Cluster 6** as being American/Fast Food/Pizza, as well as **Cluster 0** and **Cluster 2** being general neighborhoods with a large number and variety of selections.

I am comfortable with my manual entry of **Cluster 8**, **Cluster 9**, and **Cluster 10**.  When I did not, I had one or two large clusters and many clusters with only a few neighborhoods with one to four restaurants each.

I was expecting more clustering by nationality/ethnicity of cuisine.  This did not happen for a number of reasons.

First, the sheer number and variety of restaurants in the neighborhoods of **Cluster 0** and **Cluster 2** easily tell that restaurant origin is not closely correlated with a specific neighborhood.

For example, take a look at all listed Korean Restaurants in Toronto (see below). *Christie* and *Willowdale/Newtonbrook* were categorized into **Cluster 7**.  Looking at the list, one would think that *Willowdale, Willowdale East* would also be categorized into **Cluster 7**.  However, looking at all the restaurants in *Willowdale, Willowdale East*, one can see why it would be in **Cluster 0**.

The Category "Restaurant" has the largest count.  These restaurants, along with "American Restaurant", "Fast Food Restaurant", and "New American Restaurant" could be categorized in more detail.  "Asian Restaurant" is also a very broad category.

"Mexican Restaurant", "Burrito Place", "Taco Place", and "Latin American Restaurant" seem to have some overlap.  There are also  "Brazilian Restaurant", "Cuban Restaurant", and "Peruvian Restaurant", for example.  It may not be possible to have clearly defined Venue Categories, but it would assist with clustering.

Another example is "Sushi Restaurant", "Japanese Restaurant", "Ramen Restaurant", "Noodle House", and "Udon Restaurant".

Another example is "Modern European Restaurant", "German Restaurant", "Belgian Restaurant", etc.  These European restaurants could possibly be grouped into one category.

There are several others.  I think a better categorization system would lead to better clustering results.  If I split the neighborhoods further and increased the number of clusters I may also get better results.

Overall, this was an interesting exploration.

In [46]:
toronto_restaurants[toronto_restaurants['Venue Category'] == "Korean Restaurant"]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
225,Christie,43.6683,-79.4205,Sunrise House Korean Restaurant 해뜨는집,43.664183,-79.415746,Korean Restaurant
226,Christie,43.6683,-79.4205,Buk Chang Dong Soon Tofu,43.663842,-79.417093,Korean Restaurant
227,Christie,43.6683,-79.4205,Tofu Village 두부마을 豆腐村,43.663943,-79.416652,Korean Restaurant
231,Christie,43.6683,-79.4205,Yummy Korean Restaurant,43.664431,-79.413937,Korean Restaurant
234,Christie,43.6683,-79.4205,Korean Village Restaurant,43.664498,-79.414367,Korean Restaurant
236,Christie,43.6683,-79.4205,Korea House,43.664081,-79.416181,Korean Restaurant
240,Christie,43.6683,-79.4205,Doo Roo Ae 두루애,43.664175,-79.415657,Korean Restaurant
242,Christie,43.6683,-79.4205,Paldo Gangsan 팔도강산,43.663819,-79.417429,Korean Restaurant
243,Christie,43.6683,-79.4205,Lim Ga Ne Korean Restaurant 임가네,43.663945,-79.416882,Korean Restaurant
248,Christie,43.6683,-79.4205,Jangwon 장원,43.663771,-79.417713,Korean Restaurant


In [47]:
toronto_restaurants[toronto_restaurants['Neighborhood'] == "Willowdale, Willowdale East"]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
536,"Willowdale, Willowdale East",43.7673,-79.4111,Konjiki Ramen,43.766998,-79.412222,Ramen Restaurant
537,"Willowdale, Willowdale East",43.7673,-79.4111,Sushi Moto Sake & Wine Bar,43.764095,-79.411344,Sushi Restaurant
538,"Willowdale, Willowdale East",43.7673,-79.4111,Pizzaiolo,43.764289,-79.411780,Pizza Place
539,"Willowdale, Willowdale East",43.7673,-79.4111,The Captain's Boil,43.773255,-79.413805,Seafood Restaurant
540,"Willowdale, Willowdale East",43.7673,-79.4111,Sushi One,43.764412,-79.411880,Japanese Restaurant
541,"Willowdale, Willowdale East",43.7673,-79.4111,Baton Rouge,43.766455,-79.412229,American Restaurant
542,"Willowdale, Willowdale East",43.7673,-79.4111,Union Social Eatery,43.763797,-79.411358,Restaurant
543,"Willowdale, Willowdale East",43.7673,-79.4111,Wako Sushi + Bar,43.770806,-79.413138,Japanese Restaurant
544,"Willowdale, Willowdale East",43.7673,-79.4111,Shawarma Max,43.766012,-79.410844,Middle Eastern Restaurant
545,"Willowdale, Willowdale East",43.7673,-79.4111,Subway,43.768507,-79.413985,Sandwich Place
